### Locations of files

Truck OD; NHTS Zones shapefule:  https://nhts.ornl.gov/od  
FAF5: https://faf.ornl.gov/faf5/  

https://www.census.gov/data/datasets/2017/econ/cfs/historical-datasets.html  
https://www.fhwa.dot.gov/policyinformation/analysisframework/04.cfm  
https://data-usdot.opendata.arcgis.com/datasets/usdot::freight-analysis-framework-faf5-regions/about  
FIPS counties and their names:  https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt  
County lat/long:  https://gist.github.com/russellsamora/12be4f9f574e92413ea3f92ce1bc58e6  
CBSA to Counties:  https://www2.census.gov/programs-surveys/metro-micro/geographies/reference-files/2023/delineation-files/list1_2023.xlsx  
^^^ This seems to only map 1915 counties to CBSA's.
We found on https://www.fhwa.dot.gov/policyinformation/analysisframework/04.cfm a (so-called) ESRI file that maps
The 583 zones used in the Truck OD to counties
Direct link to file:  https://www.fhwa.dot.gov/policyinformation/analysisframework/zip/Revised_County_fhwa583_UMD.zip
BUT, geopandas can't seem to read that in.  But it can read just the dbf file, which is enough

US counties shapefile:  https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2022&layergroup=Counties+%28and+equivalent%29 (click on the download button) or find it: https://www2.census.gov/geo/tiger/TIGER2017/COUNTY/




In [1]:
from collections import defaultdict

import geopandas as gpd
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
import pickle

pd.options.display.max_columns = None

In [2]:
# file locations
# 
FAF5_access_db = r"C:\DocumentsOliverWyman\DOE_IntermodalRouting\Truck_Areas\FAF5_5_1_2018_2022_access.accdb"

# https://nhts.ornl.gov/od 
truck_od_dictionary = r"C:\DocumentsOliverWyman\DOE_IntermodalRouting\Truck_Areas\2022_Truck_OD_Annual_Dictionary.xlsx"
truck_od_data = r"C:\DocumentsOliverWyman\DOE_IntermodalRouting\Truck_Areas\2022_Truck_OD_Annual_Data.csv"

#  It appeared (at least a first) that the Truck OD zones were Core Based Statistical Areas (CSBA). 
#  The link below is a spreadsheet that maps CBSA to FIPS counties
#  https://www2.census.gov/programs-surveys/metro-micro/geographies/reference-files/2023/delineation-files/list1_2023.xlsx
cbsa_to_county_fips = r"C:\DocumentsOliverWyman\DOE_IntermodalRouting\Truck_Areas\CBSA_to_FIPS.xlsx"

# ^^^ This seems to only map 1915 counties to CBSA's.
# We found on https://www.fhwa.dot.gov/policyinformation/analysisframework/04.cfm a (so-called) ESRI file that maps
# The 583 zones used in the Truck OD to counties
# Direct link to file:  https://www.fhwa.dot.gov/policyinformation/analysisframework/zip/Revised_County_fhwa583_UMD.zip
# BUT, geopandas can't seem to read that in.  But it can read just the dbf file, which is enough
# Took a while to figure out how to just read the dbf file straight from the zip
# The OD_Zone_County_V2_ID 2023-10-11.dbf has the newest zones, including the new Connecticut zones
# cbsa_to_county_fips = "zip://Revised_County_fhwa583_UMD.zip!Revised_County/Revised_County.dbf"
cbsa_to_county_fips = "zip://County_version_2_ESRI.zip/OD_Zone_County_V2_ID 2023-10-11.dbf"

# computed from 01a_faf5_to_county.ipynb
faf_zone_id_to_county = r"C:\DocumentsOliverWyman\DOE_IntermodalRouting\Truck_Areas\faf_zone_id_to_state_county_id.pickle"

# https://www2.census.gov/programs-surveys/economic-census/data/2017/sector00/EC1700BASIC.zip
economic_data = "EC1700BASIC.dat"

us_counties_shape_file = "tl_2017_us_county.zip"

In [3]:
# Connecticut mapping

# Look at the Pivot worksheet in 'ct_cou_to_cousub_crosswalk.xlsx' for where the following population by old and new FIPS codes are:

ct_old_fips = ['001', '003', '005', '007', '009', '011', '013', '015']
ct_new_fips = ['110', '120', '130', '140', '150', '160', '170', '180', '190']

ct_pop_distribution = np.array([
    # new FIPS  110     120     130     140     150     160     170     180     190
            [     0, 325778,      0,  40869,      0,      0,      0,      0, 590772],  # old fips 001
            [827245,      0,      0,  60833,      0,  11420,      0,      0,      0],  # old fips 003			
            [     0,      0,      0,  54326,      0, 101083,      0,      0,  29777],  # old fips 005
		    [     0,      0, 164245,      0,      0,      0,      0,      0,      0],  # old fips 007				
			[     0,      0,      0, 294348,      0,      0, 570487,      0,      0],  # old fips 009	
		    [     0,      0,   9980,      0,   2570,      0,      0, 256005,      0],  # old fips 011	 
            [149003,      0,      0,      0,    785,      0,      0,      0,      0],  # old fips 013			
			[     0,      0,      0,      0,  91993,      0,      0,  24425,      0]   # old fips 015
            ], dtype=float)

ct_old_fips_to_idx = {fips: idx for idx, fips in enumerate(ct_old_fips)}
ct_new_fips_to_idx = {fips: idx for idx, fips in enumerate(ct_new_fips)}
ct_old_to_new_ratios = (ct_pop_distribution.T / np.sum(ct_pop_distribution, axis=1)).T
print(np.round(ct_old_to_new_ratios, 2))
ct_old_to_new_dict = {ct_old_fips[row_idx] : {ct_new_fips[col_idx]: ct_old_to_new_ratios[row_idx, col_idx] 
                                              for col_idx in range(ct_pop_distribution.shape[1])
                                              if ct_old_to_new_ratios[row_idx, col_idx] > 0.0}
                                              for row_idx in range(ct_pop_distribution.shape[0])}
print(ct_old_to_new_dict)
pd.DataFrame(ct_pop_distribution, columns = ct_new_fips, index = ct_old_fips)

[[0.   0.34 0.   0.04 0.   0.   0.   0.   0.62]
 [0.92 0.   0.   0.07 0.   0.01 0.   0.   0.  ]
 [0.   0.   0.   0.29 0.   0.55 0.   0.   0.16]
 [0.   0.   1.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.34 0.   0.   0.66 0.   0.  ]
 [0.   0.   0.04 0.   0.01 0.   0.   0.95 0.  ]
 [0.99 0.   0.   0.   0.01 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.79 0.   0.   0.21 0.  ]]
{'001': {'120': 0.34026690508544327, '140': 0.042686639809738476, '190': 0.6170464551048183}, '003': {'110': 0.9196740848784545, '140': 0.0676299447024896, '160': 0.012695970419055963}, '005': {'140': 0.2933591092199194, '160': 0.5458457982784876, '190': 0.160795092501593}, '007': {'130': 1.0}, '009': {'140': 0.34035162776714634, '170': 0.6596483722328537}, '011': {'130': 0.037161847666213624, '150': 0.009569734318854611, '180': 0.9532684180149318}, '013': {'110': 0.9947592597537853, '150': 0.00524074024621465}, '015': {'150': 0.7901956742084557, '180': 0.20980432579154426}}


,110,120,130,140,150,160,170,180,190
001,0.0,325778.0,0.0,40869.0,0.0,0.0,0.0,0.0,590772.0
003,827245.0,0.0,0.0,60833.0,0.0,11420.0,0.0,0.0,0.0
005,0.0,0.0,0.0,54326.0,0.0,101083.0,0.0,0.0,29777.0
007,0.0,0.0,164245.0,0.0,0.0,0.0,0.0,0.0,0.0
009,0.0,0.0,0.0,294348.0,0.0,0.0,570487.0,0.0,0.0
011,0.0,0.0,9980.0,0.0,2570.0,0.0,0.0,256005.0,0.0
013,149003.0,0.0,0.0,0.0,785.0,0.0,0.0,0.0,0.0
015,0.0,0.0,0.0,0.0,91993.0,0.0,0.0,24425.0,0.0


In [4]:
cbsa_to_county_fips_df = gpd.read_file(cbsa_to_county_fips)
cbsa_to_county_fips_df[cbsa_to_county_fips_df.STATEFP == '09'].sort_values(by='COUNTYFP')

# eliminate Puerto Rico (State FIP 72), Virgin Islands (78), Northern Mariana Islands (69), Guam (66), America Somoa (60)
cbsa_to_county_fips_df = cbsa_to_county_fips_df[~cbsa_to_county_fips_df.STATEFP.isin(['72', '78', '69', '66', '60'])]

# for whatever reason, Aransas County, TX (FIPS: 48007) does not have a CBSA code.
# However, according to https://www.uspto.gov/web/offices/ac/ido/oeip/taf/cls_cbsa/cbsa_countyassoc.htm, 48007 belongs
# to the Corpus Christi TX CBSA, 18580_TX
cbsa_to_county_fips_df.loc[(cbsa_to_county_fips_df.STATEFP == '48') & (cbsa_to_county_fips_df.COUNTYFP == '007'), 'V1ZONEID'] = '18580_TX'

In [5]:
pd.DataFrame(cbsa_to_county_fips_df).to_csv('c:/temp/cbsa.dat', index=False)

In [6]:
cbsa_to_county_fips_df[pd.isna(cbsa_to_county_fips_df.V2ZONEID)]    # .str.startswith('16980')

,ID,AREA,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,V1ZONENAME,V1ZONEID,V2ZONEID,geometry
717,1073,313.061951,48,007,01383789,0500000US48007,48007,Aransas,Aransas County,TX,Texas,NaN,18580_TX,NaN,"MULTIPOLYGON (((-97.13480 27.90247, -97.15440 ..."


In [7]:
economic_data_df = pd.read_csv(economic_data, sep="|", dtype={"ST": str, "COUNTY": str, "CENREG": object, "GEOCOMP": str, "PLACE": str,
                                                              "CSA": str, "MSA": str, "MD": str, 
                                                              "NAICS2017": str, "TAXSTAT": str, "FIRM_F": str})

In [8]:
# naics_to_be_used = {"21", "23", "42", "31-33", "44-45", "48-49"}
# NAICS 21 and 23 are only on the state level, and 42 is only on a 'micro' or 'metro' level but not a county level
naics_to_be_used = {"31-33", "44-45", "48-49"}
[(n,c) for n, c in economic_data_df.NAICS2017.value_counts().items() if n in naics_to_be_used]

[('44-45', 19451), ('48-49', 15617), ('31-33', 4164)]

In [9]:
# Employment flag decoder.  Pick halfway point for small values, 20% higher than min for large values
emp_f_decoder = {"a": ("0 to 19 employees", 10.0),
                 "b": ("20 to 99 employees", 50.0),
                 "c": ("100 to 249 employees", 175.0),
                 "e": ("250 to 499 employees", 375.0),
                 "f": ("500 to 999 employees", 600.0),
                 "g": ("1,000 to 2,499 employees", 1200.0),
                 "h": ("2,500 to 4,999 employees", 3000.0),
                 "i": ("5,000 to 9,999 employees", 6000.0),
                 "j": ("10,000 to 24,999 employees", 12000.0),
                 "k": ("25,000 to 49,999 employees", 30000.0),
                 "l": ("50,000 to 99,999 employees", 60000.0),
                 "m": ("100,000 employees or more", 120000.0)}

# Receipts flag decoder.  Data is in units of $1000.  Turns out that only "D" is used, sigh
rcptot_f_decoder = {
    "D" : ("Withheld to avoid disclosing data for individual companies; data are included in higher level totals", 0.0),
    "B" : ("Less than $1 million", 200.0),
	"I" : ("$1 million to less than $5 million", 2500.0),
	"K" : ("$5 million to less than $15 million", 7500.0),
	"L" : ("$15 million to less than $50 million", 20000.0),
	"M" : ("$50 million to less than $75 million", 60000.0),
	"O" : ("$75 million to less than $150 million", 90000.0),
	"R" : ("$150 million to less than $500 million", 180000.0),
	"T" : ("$500 million to less than $1 billion", 600000.0),
	"U" : ("$1 billion to less than $5 billion", 1200000.0),
	"W" : ("$5 billion or more", 6000000.0)
}

In [10]:
state_ec_df = economic_data_df[
    (economic_data_df.NAICS2017.isin(naics_to_be_used)) &
    (economic_data_df.GEOTYPE == 2) &       # GEOTYPE 2 is state level
    # CSA 000 eliminates some 'not in metropolitan or micropolitan'
    (economic_data_df.CSA == '000') &
    (economic_data_df.TAXSTAT == "00") &   # All Establishments
    (economic_data_df.TYPOP == 0)          # All Establishments
].drop_duplicates().sort_values(by="NAICS2017")

In [11]:

census_ec_df = economic_data_df[
                    # (economic_data_df.NAICS2017.str.startswith("48") | economic_data_df.NAICS2017.str.startswith("49")) & 
                    # (economic_data_df.NAICS2017.str.startswith("23")) & 
                    (economic_data_df.NAICS2017.isin(naics_to_be_used)) & 
                    # (economic_data_df.NAICS2017.isin({"42"})) & 
                    # (economic_data_df.ST == "42") & 
                    # (economic_data_df.COUNTY == "133") & 
                    (economic_data_df.GEOTYPE == 3) &       # GEOTYPE 3 is county level
                     (economic_data_df.TAXSTAT == "00") &   # All Establishments
                     (economic_data_df.TYPOP == 0)          # All Establishments
                     ].drop_duplicates().sort_values(by="NAICS2017")

In [12]:
# find average receipts by employee, by state and naics
receipts_per_employee: dict[tuple[str, str], float] = dict()
total_receipts_by_state: defaultdict[str, float] = defaultdict(float)
for row in state_ec_df.itertuples(index=False):
    state = str(row.ST)
    naics = str(row.NAICS2017)
    ratio = float(row.RCPTOT)/float(row.EMP)
    receipts_per_employee[state, naics] = ratio
    total_receipts_by_state[state] += float(row.RCPTOT)
total_receipts_by_state

defaultdict(float,
            {'01': 210171788.0,
             '15': 34146271.0,
             '44': 27626471.0,
             '16': 48257400.0,
             '17': 466326609.0,
             '41': 132923584.0,
             '18': 369163687.0,
             '40': 127223997.0,
             '19': 168704377.0,
             '39': 509982956.0,
             '20': 130595378.0,
             '38': 37223159.0,
             '21': 212088350.0,
             '37': 358338552.0,
             '22': 267024018.0,
             '36': 497311812.0,
             '23': 40813866.0,
             '35': 42944150.0,
             '24': 138057381.0,
             '34': 275556839.0,
             '25': 206099056.0,
             '33': 51927063.0,
             '26': 430952271.0,
             '32': 69172747.0,
             '27': 232349539.0,
             '31': 91827570.0,
             '28': 103391136.0,
             '30': 30031128.0,
             '45': 216134159.0,
             '13': 353248027.0,
             '42': 491359042.0,

In [13]:
# # find average receipts by employee
# receipts_per_employee = census_ec_df[pd.isna(census_ec_df.EMP_F) & 
#                                      pd.isna(census_ec_df.RCPTOT_F)].groupby(census_ec_df.NAICS2017).agg({'ST': len, 'EMP': sum, 'RCPTOT': sum})
# receipts_per_employee["RCP_PER_EMP"] = receipts_per_employee.RCPTOT / receipts_per_employee.EMP
# receipts_per_employee.rename(columns={'ST': 'COUNT'}, inplace=True)
# receipts_per_employee

In [14]:
all_receipts: defaultdict[tuple[str, str, str], float] = defaultdict(float)
all_employees: defaultdict[tuple[str, str, str], float] = defaultdict(float)
counties_by_state_naics_with_estimated_receipts: defaultdict[tuple[str, str], set[str]] = defaultdict(set)
counties_by_state_naics_with_receipts: defaultdict[tuple[str, str], set[str]] = defaultdict(set)
total_estimated_receipts_for_D_counties: defaultdict[str, float] = defaultdict(float)
for row in census_ec_df.itertuples(index=True):
    state = str(row.ST)
    county = str(row.COUNTY)
    naics = str(row.NAICS2017)
    
    if pd.isna(row.EMP_F):
        number_employees = float(row.EMP)
    elif row.EMP_F in emp_f_decoder:
        number_employees = emp_f_decoder[row.EMP_F][1]
    else:
        # from what I can tell, will never reach here
        number_employees = 0
    
    if pd.isna(row.RCPTOT_F):
        receipts = float(row.RCPTOT)
        counties_by_state_naics_with_receipts[state, naics].add(county)        
    else:
        counties_by_state_naics_with_estimated_receipts[state, naics].add(county)
        if row.RCPTOT_F == "D":
            receipts_per_emp = receipts_per_employee[state, naics]
            receipts = number_employees * receipts_per_emp
            total_estimated_receipts_for_D_counties[state] += receipts
            if state == '36':
                pass
        elif row.RCPTOT_F in rcptot_f_decoder:
            receipts = rcptot_f_decoder[row.RCPTOT_F][1]
        else:
            # should not reach here
            receipts = 0.0

    
    all_employees[state, naics, county] += number_employees
    all_receipts[state, naics, county] += receipts

# adjust receipts for the counties with estimated receipts so that
# the sum of all counties in the state is the state total
for state in total_receipts_by_state.keys():
    true_state_receipts = total_receipts_by_state[state]
    known_state_county_receipts = sum(all_receipts[state, naics, county] 
                                      for naics in naics_to_be_used 
                                      for county in counties_by_state_naics_with_receipts[state, naics])
    estimated_state_county_receipts = sum(all_receipts[state, naics, county] 
                                          for naics in naics_to_be_used
                                          for county in counties_by_state_naics_with_estimated_receipts[state, naics])
    diff = true_state_receipts - known_state_county_receipts
    ratio = 0.0 if estimated_state_county_receipts == 0.0 else diff / estimated_state_county_receipts
    print(f'STATEFP: {state}, true receipts: {true_state_receipts:0.0f}, '
          f'known: {known_state_county_receipts:0.0f}, '
          f'estimated: {estimated_state_county_receipts:0.0f}, '
          f'true-known: {diff:0.0f}, '
          f'ratio (true-known)/estimated: {ratio:0.3f}')
    
    if diff < 0:
        pass
    elif diff > 0.0:
        for naics in naics_to_be_used:
            for county in counties_by_state_naics_with_estimated_receipts[state, naics]:
                all_receipts[state, naics, county] *= ratio

all_receipts_list: list[float] = list()
all_employees_list: list[float] = list()
for row in census_ec_df.itertuples(index=False):
    state = str(row.ST)
    county = str(row.COUNTY)
    all_receipts_list.append(sum(all_receipts[state, naics, county] for naics in naics_to_be_used if (state, naics, county) in all_receipts))
    all_employees_list.append(sum(all_employees[state, naics, county] for naics in naics_to_be_used if (state, naics, county) in all_receipts))
census_ec_df['ALL_RECEIPTS'] = all_receipts_list
census_ec_df['ALL_EMPLOYEES'] = all_employees_list
    
census_ec_df.head()

STATEFP: 01, true receipts: 210171788, known: 208063807, estimated: 1450203, true-known: 2107981, ratio (true-known)/estimated: 1.454
STATEFP: 15, true receipts: 34146271, known: 31092581, estimated: 1782431, true-known: 3053690, ratio (true-known)/estimated: 1.713
STATEFP: 44, true receipts: 27626471, known: 27626471, estimated: 0, true-known: 0, ratio (true-known)/estimated: 0.000
STATEFP: 16, true receipts: 48257400, known: 47038863, estimated: 990568, true-known: 1218537, ratio (true-known)/estimated: 1.230
STATEFP: 17, true receipts: 466326609, known: 453112820, estimated: 6390250, true-known: 13213789, ratio (true-known)/estimated: 2.068
STATEFP: 41, true receipts: 132923584, known: 132854361, estimated: 172143, true-known: 69223, ratio (true-known)/estimated: 0.402
STATEFP: 18, true receipts: 369163687, known: 367359896, estimated: 1967592, true-known: 1803791, ratio (true-known)/estimated: 0.917
STATEFP: 40, true receipts: 127223997, known: 123499154, estimated: 4495416, true-k

,GEOTYPE,ST,COUNTY,PLACE,CONSCITY,CSA,MSA,MD,CENREG,GEOCOMP,GEO_ID,GEO_TTL,GEO_ID_F,NAICS2017,NAICS2017_TTL,NAICS2017_F,TYPOP,TYPOP_TTL,TAXSTAT,TAXSTAT_TTL,YEAR,FIRM,FIRM_F,ESTAB,ESTAB_F,RCPTOT,RCPTOT_F,PAYANN,PAYANN_F,PAYQTR1,PAYQTR1_F,EMP,EMP_F,RCPTOT_IMP,RCPTOT_IMP_F,PAYANN_IMP,PAYANN_IMP_F,EMP_IMP,EMP_IMP_F,ALL_RECEIPTS,ALL_EMPLOYEES
178570,3,01,001,00000,NaN,999,33860,99999,3,00,0500000US01001,"Autauga County, Alabama",NaN,31-33,Manufacturing,NaN,0,All establishments,00,All establishments,2017,23,NaN,23,NaN,760106,NaN,79721,NaN,18410,NaN,1152,NaN,0,1,0,1,0,2,1.474214e+06,3998.0
747536,3,29,061,00000,NaN,999,99999,99999,2,00,0500000US29061,"Daviess County, Missouri",NaN,31-33,Manufacturing,NaN,0,All establishments,00,All establishments,2017,9,NaN,9,NaN,52200,NaN,5979,NaN,1324,NaN,170,NaN,0,0,0,0,0,0,1.195795e+05,396.0
747610,3,29,063,00000,NaN,312,41140,99999,2,00,0500000US29063,"DeKalb County, Missouri",NaN,31-33,Manufacturing,NaN,0,All establishments,00,All establishments,2017,6,NaN,6,NaN,7765,NaN,1407,NaN,0,D,25,NaN,0,2,0,0,0,5,1.534870e+05,652.0
747719,3,29,065,00000,NaN,999,99999,99999,2,00,0500000US29065,"Dent County, Missouri",NaN,31-33,Manufacturing,NaN,0,All establishments,00,All establishments,2017,25,NaN,25,NaN,291332,NaN,31886,NaN,7886,NaN,556,NaN,0,1,0,1,0,1,4.242500e+05,1150.0
747869,3,29,067,00000,NaN,999,99999,99999,2,00,0500000US29067,"Douglas County, Missouri",NaN,31-33,Manufacturing,NaN,0,All establishments,00,All establishments,2017,11,NaN,11,NaN,122126,NaN,17092,NaN,0,D,0,f,0,0,0,0,0,D,2.269110e+05,1076.0


In [15]:
county_receipts = {(st_fips, county_fips): receipts 
                   for st_fips, county_fips, receipts in 
                   list(census_ec_df.groupby(by=['ST', 'COUNTY'])[['ALL_RECEIPTS']]
                        .sum()
                        .reset_index()
                        .itertuples(index=False, name=None))}
county_receipts

# transform for Connecticut
for county_fips in ct_new_fips:
    county_receipts[("09", county_fips)] = 0.0
for county_old_fips in ct_old_fips:
    receipts = county_receipts["09", county_old_fips]
    for county_new_fips, ratio in ct_old_to_new_dict[county_old_fips].items():
        county_receipts["09", county_new_fips] += receipts * ratio
    del county_receipts["09", county_old_fips]
county_receipts

{('01', '001'): 4422642.0,
 ('01', '003'): 19487592.0,
 ('01', '005'): 3501045.0,
 ('01', '007'): 874114.2314044229,
 ('01', '009'): 2136429.0,
 ('01', '011'): 1093093.7168503483,
 ('01', '013'): 3368628.0,
 ('01', '015'): 13607214.0,
 ('01', '017'): 3646980.0,
 ('01', '019'): 1661685.0,
 ('01', '021'): 3104190.0,
 ('01', '023'): 1845447.8066799454,
 ('01', '025'): 3073734.0,
 ('01', '027'): 1109544.351829835,
 ('01', '029'): 803073.0,
 ('01', '031'): 5248116.0,
 ('01', '033'): 13859865.0,
 ('01', '035'): 859641.0,
 ('01', '037'): 317742.0,
 ('01', '039'): 3635160.0,
 ('01', '041'): 1187484.0,
 ('01', '043'): 10369971.0,
 ('01', '045'): 2978445.0,
 ('01', '047'): 4368762.0,
 ('01', '049'): 6554775.0,
 ('01', '051'): 5076801.0,
 ('01', '053'): 3736464.0,
 ('01', '055'): 9242922.0,
 ('01', '057'): 945048.351829835,
 ('01', '059'): 5475711.0,
 ('01', '061'): 963954.0,
 ('01', '063'): 550155.0,
 ('01', '065'): 682200.0,
 ('01', '067'): 1014915.0,
 ('01', '069'): 11561343.0,
 ('01', '071'):

In [16]:
connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=" + FAF5_access_db + ";"
    r"ExtendedAnsiSQL=1;"
)
connection_url = sa.engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)
engine = sa.create_engine(connection_url)

In [17]:
with engine.connect() as conn:
    mode_df = pd.read_sql("Mode", con=conn)
    trade_df = pd.read_sql("Trade", conn)
    state_df = pd.read_sql("State", conn)
    foreign_zone_df = pd.read_sql("Foreign_Zone", conn)
    faf_zone_df = pd.read_sql("FAF_Zone",conn)
    faf = pd.read_sql("FAF551", conn)

print(f'mode_df.shape:         {mode_df.shape}')
print(f'trade_df.shape:        {trade_df.shape}')
print(f'state_df.shape:        {state_df.shape}')
print(f'foreign_zone_df.shape: {foreign_zone_df.shape}')
print(f'faf_zone_df.shape:     {faf_zone_df.shape}')
print(f'faf.shape:             {faf.shape}')

mode_df.shape:         (8, 2)
trade_df.shape:        (3, 2)
state_df.shape:        (51, 2)
foreign_zone_df.shape: (8, 2)
faf_zone_df.shape:     (132, 3)
faf.shape:             (2311632, 30)


In [18]:
truck_od_dict = pd.read_excel(truck_od_dictionary, sheet_name=None)
truck_od_data_df = pd.read_csv(truck_od_data)
truck_od_data_df['origin_cbsa'] = truck_od_data_df.origin_zone_id.str[:5]
truck_od_data_df['destination_cbsa'] = truck_od_data_df.destination_zone_id.str[:5]

In [19]:
truck_od_scheme_df = truck_od_dict["National Truck OD Data Schema"]
national_zones_df = truck_od_dict["National Zone Lookup Table"]

In [20]:
truck_od_data_df.head()

,year,origin_zone_id,destination_zone_id,origin_zone_name,destination_zone_name,origin_state,destination_state,annual_total_trips,distance_0_10,distance_10_25,distance_25_50,distance_50_75,distance_75_100,distance_100_150,distance_150_300,distance_gt300,distance_0_10_pct,distance_10_25_pct,distance_25_50_pct,distance_50_75_pct,distance_75_100_pct,distance_100_150_pct,distance_150_300_pct,distance_gt300_pct,origin_cbsa,destination_cbsa
0,2022,10180_TX,10180_TX,"Abilene, TX","Abilene, TX",TX,TX,2610745,1807753,609957,182982,9562,458,28,5,0,0.692428,0.233633,0.070088,0.003663,0.000175,0.000011,0.000002,0.0,10180,10180
1,2022,10420_OH,10180_TX,"Akron, OH","Abilene, TX",OH,TX,36,0,0,0,0,0,0,0,36,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,10420,10180
2,2022,10500_GA,10180_TX,"Albany, GA","Abilene, TX",GA,TX,37,0,0,0,0,0,0,0,37,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,10500,10180
3,2022,10540_OR,10180_TX,"Albany, OR","Abilene, TX",OR,TX,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,10540,10180
4,2022,10580_NY,10180_TX,"Albany-Schenectady-Troy, NY","Abilene, TX",NY,TX,15,0,0,0,0,0,0,0,15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,10580,10180


In [21]:
with open(faf_zone_id_to_county, 'rb') as faf_file:
    faf_to_fips = pickle.load(faf_file)
faf_to_fips

{'361': {('36', '091'): 1.0,
  ('36', '001'): 1.0,
  ('36', '113'): 1.0,
  ('36', '115'): 1.0,
  ('36', '057'): 1.0,
  ('36', '095'): 1.0,
  ('36', '083'): 1.0,
  ('36', '021'): 1.0,
  ('36', '035'): 1.0,
  ('36', '093'): 1.0},
 '131': {('13', '171'): 1.0,
  ('13', '063'): 1.0,
  ('13', '233'): 1.0,
  ('13', '089'): 1.0,
  ('13', '227'): 1.0,
  ('13', '045'): 1.0,
  ('13', '297'): 1.0,
  ('13', '129'): 1.0,
  ('13', '293'): 1.0,
  ('13', '013'): 1.0,
  ('13', '223'): 1.0,
  ('13', '199'): 1.0,
  ('13', '113'): 1.0,
  ('13', '143'): 1.0,
  ('13', '015'): 1.0,
  ('13', '139'): 1.0,
  ('13', '059'): 1.0,
  ('13', '077'): 1.0,
  ('13', '159'): 1.0,
  ('13', '151'): 1.0,
  ('13', '085'): 1.0,
  ('13', '097'): 1.0,
  ('13', '211'): 1.0,
  ('13', '135'): 1.0,
  ('13', '231'): 1.0,
  ('13', '285'): 1.0,
  ('13', '247'): 1.0,
  ('13', '121'): 1.0,
  ('13', '221'): 1.0,
  ('13', '149'): 1.0,
  ('13', '219'): 1.0,
  ('13', '195'): 1.0,
  ('13', '067'): 1.0,
  ('13', '057'): 1.0,
  ('13', '217'): 

In [22]:
# How many counties are there?

census_data_counties = economic_data_df[economic_data_df.GEOTYPE == 3][['ST', 'COUNTY']].drop_duplicates().sort_values(by=['ST', 'COUNTY'])
census_data_counties.head()
print(f'Number of counties in original census data: {census_data_counties.shape[0]}')

census_ec_counties = census_ec_df[census_ec_df.GEOTYPE == 3][['ST', 'COUNTY']].drop_duplicates().sort_values(by=['ST', 'COUNTY'])
print(f'Number of counties in naics restricted census data: {census_ec_counties.shape[0]}')

cbsa_counties = frozenset(cbsa_to_county_fips_df[['STATEFP', 'COUNTYFP']].drop_duplicates().itertuples(index=False, name=None))
print(f'Number of counties in cbsa to fips data: {len(cbsa_counties)}')

faf5_counties = frozenset(county for county_proportions in faf_to_fips.values() for county in county_proportions.keys())
print(f'Number of counties in faf5 to fips data: {len(faf5_counties)}')

faf5_counties_not_in_cbsa_counties = faf5_counties - cbsa_counties
if len(faf5_counties_not_in_cbsa_counties) == 0:
    print('All faf5 counties are in cbsa counties')
else:
    print(f'There are {len(faf5_counties_not_in_cbsa_counties)} faf5 counties not in cbsa counties: {sorted(faf5_counties_not_in_cbsa_counties)}')

cbsa_counties_not_in_faf5_counties = cbsa_counties - faf5_counties
if len(cbsa_counties_not_in_faf5_counties) == 0:
    print('All cbsa counties are in faf5 counties')
else:
    print(f'There are {len(cbsa_counties_not_in_faf5_counties)} cbsa counties not in faf5 counties: {sorted(cbsa_counties_not_in_faf5_counties)}')

cbsa_in_cbsa_counties = frozenset(cbsa_to_county_fips_df['V1ZONEID'])
cbas_in_truck_od = frozenset(truck_od_data_df['origin_zone_id'])

cbsa_in_county_not_in_truck_od = cbsa_in_cbsa_counties - cbas_in_truck_od
cbsa_in_truck_od_not_in_county = cbas_in_truck_od - cbsa_in_cbsa_counties
if len(cbsa_in_county_not_in_truck_od) == 0:
    print("All truck OD cbsa's are in the cbsa_county map")
else:
    print("Investigate, {len(cbsa_in_county_not_in_truck_od)} entries in the cbsa_in_cbsa_counties not found in the truck OD data")

if len(cbsa_in_truck_od_not_in_county) == 0:
    print("All cbsa's in the cbsa_in_cbas_counties are in the truck ods")
else:
    print("Investigate, {len(cbsa_in_truck_od_not_in_county)} entries in the truck OD data are not found in the cbsa_in_cbsa_counties")    


Number of counties in original census data: 3132
Number of counties in naics restricted census data: 3125
Number of counties in cbsa to fips data: 3144
Number of counties in faf5 to fips data: 3144
All faf5 counties are in cbsa counties
All cbsa counties are in faf5 counties
All truck OD cbsa's are in the cbsa_county map
All cbsa's in the cbsa_in_cbas_counties are in the truck ods


In [23]:
# Spread Truck OD into a county-by-county specific, using receipts by county as the proportional factor

# Start with creating a Numpy matrix of the truck trips by CBSA

idx_to_cbsa = list(truck_od_data_df['origin_zone_id'].drop_duplicates())
print(f'Number of truck OD zones: {len(idx_to_cbsa)}')
cbsa_to_idx = {str(cbsa): idx for idx, cbsa in enumerate(idx_to_cbsa)}

truck_od_np = np.zeros([len(idx_to_cbsa), len(idx_to_cbsa)])

for row in truck_od_data_df.itertuples(index=False):
    i = cbsa_to_idx[row.origin_zone_id]
    j = cbsa_to_idx[row.destination_zone_id]
    truck_od_np[i, j] = row.annual_total_trips

Number of truck OD zones: 583


In [24]:
# Calculate cbsa, county receipt percentage

cbsa_to_county_fips: defaultdict[str, list[tuple[str, str]]] = defaultdict(list)
county_fips_to_cbas: dict[tuple[str, str], str] = dict()
for row in cbsa_to_county_fips_df.itertuples(index=False):
    cbsa_to_county_fips[row.V1ZONEID].append((row.STATEFP, row.COUNTYFP))
    county_fips_to_cbas[(row.STATEFP, row.COUNTYFP)] = row.V1ZONEID

cbsa_total_receipts: defaultdict[str, float] = defaultdict(float)
for st_county_fips, receipts in county_receipts.items():
    if st_county_fips in county_fips_to_cbas:
        cbsa = county_fips_to_cbas[st_county_fips]
        cbsa_total_receipts[cbsa] += receipts
    else:
        print(f'FIPS {st_county_fips} is not in cbsa counties')

cbsa_to_county_fips_ratios: dict[str, list[tuple[tuple[str, str], float]]] = dict()
for cbsa, counties in cbsa_to_county_fips.items():
    # if cbsa not in cbsa_to_county_fips_ratios:
    #     print(f'not found: {cbsa}, {cbsa_to_county_fips[cbsa]}')
    #     continue
    total_receipts = cbsa_total_receipts[cbsa]
    cbsa_to_county_fips_ratios[cbsa]: list[tuple[tuple[str, str], float]] = list()
    for county in counties:
        if county in county_receipts:
            cbsa_to_county_fips_ratios[cbsa].append((county, county_receipts[county] / total_receipts))
        else:
            print(f'county {county} not in county_receipts')
            cbsa_to_county_fips_ratios[cbsa].append((county, 0.0))
cbsa_to_county_fips_ratios

FIPS ('02', '261') is not in cbsa counties
county ('48', '393') not in county_receipts
county ('48', '269') not in county_receipts
county ('48', '033') not in county_receipts
county ('02', '063') not in county_receipts
county ('02', '066') not in county_receipts
county ('48', '261') not in county_receipts
county ('16', '033') not in county_receipts
county ('15', '005') not in county_receipts
county ('31', '009') not in county_receipts
county ('31', '005') not in county_receipts
county ('31', '117') not in county_receipts
county ('31', '007') not in county_receipts
county ('48', '301') not in county_receipts
county ('48', '011') not in county_receipts
county ('46', '017') not in county_receipts
county ('30', '037') not in county_receipts
county ('28', '055') not in county_receipts
county ('06', '003') not in county_receipts
county ('32', '009') not in county_receipts


{'16980_IL': [(('17', '031'), 0.5653263930423263),
  (('17', '097'), 0.08037074404656004),
  (('17', '111'), 0.02762290282757435),
  (('17', '089'), 0.06516640630246846),
  (('17', '093'), 0.0075603431968663095),
  (('17', '043'), 0.14791965962384487),
  (('17', '197'), 0.08858714804613554),
  (('17', '063'), 0.009444640958779983),
  (('17', '037'), 0.00800176195544406)],
 '16980_WI': [(('55', '059'), 1.0)],
 'RMT2_MT': [(('30', '051'), 0.0035372115370551765),
  (('30', '101'), 0.05477115371845447),
  (('30', '035'), 0.07818995641233215),
  (('30', '107'), 0.00843796116414055),
  (('30', '097'), 0.030658695900741374),
  (('30', '099'), 0.03403361653162599),
  (('30', '059'), 0.002094471420413973),
  (('30', '041'), 0.13234236353065132),
  (('30', '007'), 0.11687035699065429),
  (('30', '073'), 0.06008122583790608),
  (('30', '067'), 0.13958089048599434),
  (('30', '027'), 0.11646613276009277),
  (('30', '045'), 0.0069404543489936694),
  (('30', '095'), 0.1978329102432018),
  (('30', '0

In [25]:
{cbsa: sum(ratios for _, ratios in counties_ratios) for cbsa, counties_ratios in cbsa_to_county_fips_ratios.items()}

{'16980_IL': 1.0,
 '16980_WI': 1.0,
 'RMT2_MT': 1.0000000000000002,
 'RSD1_SD': 0.9999999999999998,
 'RTX2_TX': 0.9999999999999996,
 '33260_TX': 1.0,
 'RMS1_MS': 1.0000000000000002,
 'RMS5_MS': 1.0,
 'RND2_ND': 1.0,
 'RAK1_AK': 1.0,
 'RIA1_IA': 0.9999999999999998,
 'RNE2_NE': 1.0,
 '19780_IA': 1.0,
 'RIA3_IA': 1.0,
 'RKS3_KS': 1.0,
 'RKS2_KS': 0.9999999999999999,
 '25180_MD': 1.0,
 '19060_MD': 1.0,
 'RTX7_TX': 1.0,
 'RMI3_MI': 0.9999999999999999,
 '33100_FL': 1.0,
 '37860_FL': 1.0,
 '16580_IL': 1.0,
 '28100_IL': 1.0,
 '16980_IN': 1.0,
 'RMN3_MN': 1.0,
 '24220_MN': 1.0,
 'ROK3_OK': 1.0,
 'RMT3_MT': 1.0000000000000002,
 'RID2_ID': 1.0000000000000002,
 'RMO3_MO': 0.9999999999999999,
 'RMS7_MS': 1.0,
 'RAL1_AL': 1.0,
 'RMO2_MO': 1.0,
 'RMN1_MN': 1.0,
 'RIA2_IA': 0.9999999999999999,
 '27980_HI': 1.0,
 'RME2_ME': 1.0,
 '27860_AR': 1.0,
 'RAR1_AR': 1.0,
 'RNE1_NE': 1.0,
 'RKS4_KS': 1.0000000000000002,
 'RIL1_IL': 1.0000000000000002,
 'RKY1_KY': 1.0,
 '37900_IL': 1.0,
 'RIL2_IL': 1.0,
 '12980_

In [26]:
# Spread truck trips (from the truck_od_data_df) to county using the census bureau economic receipts data.  Use the

# so far, we have established that all the counties associated with CSBA and with FAF5 are the same
# but just in case, we take the union on the cbsa and faft5 counties
idx_to_county: list[tuple[str, str]] = list(sorted(cbsa_counties | faf5_counties))
county_to_idx: dict[tuple[str, str], int] = {county: idx for idx, county in enumerate(idx_to_county)}
from_cbsas = list()
to_cbsas = list()
from_st_fips: list[str] = list()
from_county_fips: list[str] = list()
to_st_fips: list[str] = list()
to_county_fips: list[str] = list()
cbsa_truck_trips: list[float] = list()
county_truck_trips: list[float] = list()

for row in truck_od_data_df.itertuples(index=False):
    from_cbsa = str(row.origin_zone_id)
    to_cbsa = str(row.destination_zone_id)
    truck_trips_annual = row.annual_total_trips
    for from_county_fip, from_ratio in cbsa_to_county_fips_ratios[from_cbsa]:
        for to_county_fip, to_ratio in cbsa_to_county_fips_ratios[to_cbsa]:
            from_cbsas.append(from_cbsa)
            to_cbsas.append(to_cbsa)
            from_st_fips.append(from_county_fip[0])
            from_county_fips.append(from_county_fip[1])
            to_st_fips.append(to_county_fip[0])
            to_county_fips.append(to_county_fip[1])
            cbsa_truck_trips.append(truck_trips_annual)
            county_truck_trips_annual = from_ratio * to_ratio * truck_trips_annual
            county_truck_trips.append(county_truck_trips_annual)

truck_by_county_od_df = pd.DataFrame({
    'FROM_CBSA': from_cbsas,
    'TO_CBSA': to_cbsas,
    'FROM_ST_FIPS': from_st_fips,
    'FROM_COUNTY_FIPS': from_county_fips,
    'TO_ST_FIPS': to_st_fips,
    'TO_COUNTY_FIPS': to_county_fips,
    'CBSA_ANNUAL_TRUCK_TRIPS': cbsa_truck_trips,
    'COUNTY_TRUCK_TRIPS': county_truck_trips
}) 
truck_by_county_od_df

,FROM_CBSA,TO_CBSA,FROM_ST_FIPS,FROM_COUNTY_FIPS,TO_ST_FIPS,TO_COUNTY_FIPS,CBSA_ANNUAL_TRUCK_TRIPS,COUNTY_TRUCK_TRIPS
0,10180_TX,10180_TX,48,441,48,441,2610745,1.901699e+06
1,10180_TX,10180_TX,48,441,48,059,2610745,1.690913e+05
2,10180_TX,10180_TX,48,441,48,253,2610745,1.574041e+05
3,10180_TX,10180_TX,48,059,48,441,2610745,1.690913e+05
4,10180_TX,10180_TX,48,059,48,059,2610745,1.503491e+04
...,...,...,...,...,...,...,...,...
9884731,RWY3_WY,RWY3_WY,56,035,56,037,2673319,1.115467e+05
9884732,RWY3_WY,RWY3_WY,56,037,56,041,2673319,2.573296e+05
9884733,RWY3_WY,RWY3_WY,56,037,56,023,2673319,2.523972e+05
9884734,RWY3_WY,RWY3_WY,56,037,56,035,2673319,1.115467e+05


In [27]:
print(f"truck_by_county_od_df.shape: {truck_by_county_od_df.shape[0]}, "
      f"truck_by_county_od_df[['FROM_ST_FIPS', 'FROM_COUNTY_FIPS', 'TO_ST_FIPS', 'TO_COUNTY_FIPS']].drop_duplicates(): "
      f"{truck_by_county_od_df[['FROM_ST_FIPS', 'FROM_COUNTY_FIPS', 'TO_ST_FIPS', 'TO_COUNTY_FIPS']].drop_duplicates().shape[0]}")

truck_by_county_od_df.shape: 9884736, truck_by_county_od_df[['FROM_ST_FIPS', 'FROM_COUNTY_FIPS', 'TO_ST_FIPS', 'TO_COUNTY_FIPS']].drop_duplicates(): 9884736


In [28]:
od_fips_to_truck_trips: dict[tuple[tuple[str, str], tuple[str, str]], float] = {((row.FROM_ST_FIPS, row.FROM_COUNTY_FIPS), (row.TO_ST_FIPS, row.TO_COUNTY_FIPS)): row.COUNTY_TRUCK_TRIPS
                                                                                for row in  truck_by_county_od_df[['FROM_ST_FIPS', 'FROM_COUNTY_FIPS', 'TO_ST_FIPS', 'TO_COUNTY_FIPS', 'COUNTY_TRUCK_TRIPS']].itertuples(index=False)
}
od_fips_to_truck_trips

{(('48', '441'), ('48', '441')): 1901699.4276642667,
 (('48', '441'), ('48', '059')): 169091.3080592833,
 (('48', '441'), ('48', '253')): 157404.1103968124,
 (('48', '059'), ('48', '441')): 169091.3080592833,
 (('48', '059'), ('48', '059')): 15034.90511974175,
 (('48', '059'), ('48', '253')): 13995.72747076815,
 (('48', '253'), ('48', '441')): 157404.1103968124,
 (('48', '253'), ('48', '059')): 13995.72747076815,
 (('48', '253'), ('48', '253')): 13028.375362263594,
 (('39', '133'), ('48', '441')): 7.383948543332581,
 (('39', '133'), ('48', '059')): 0.656550398907188,
 (('39', '133'), ('48', '253')): 0.6111711634191512,
 (('39', '153'), ('48', '441')): 23.34100332302474,
 (('39', '153'), ('48', '059')): 2.0753862181858467,
 (('39', '153'), ('48', '253')): 1.9319403531304888,
 (('13', '007'), ('48', '441')): 0.02070477129902954,
 (('13', '007'), ('48', '059')): 0.0018409832863657413,
 (('13', '007'), ('48', '253')): 0.0017137388063980415,
 (('13', '321'), ('48', '441')): 2.58159446911389

In [29]:
truck_by_county_od_df.groupby(by=['FROM_CBSA', 'TO_CBSA'])['COUNTY_TRUCK_TRIPS'].sum()

FROM_CBSA  TO_CBSA 
10180_TX   10180_TX    2610745.0
           10420_OH         18.0
           10500_GA         38.0
           10540_OR          5.0
           10580_NY          0.0
                         ...    
RWY3_WY    RWV2_WV          21.0
           RWV3_WV           0.0
           RWY1_WY      570564.0
           RWY2_WY      150745.0
           RWY3_WY     2673319.0
Name: COUNTY_TRUCK_TRIPS, Length: 339889, dtype: float64

### So now we have a county to county spread of the truck trips.

Use this to spread out FAF data

Note, for now at least, we will use the same factors for every FAF OD.  So let's pre-calculate the OD

In [31]:
faf_ods = set(faf[['dms_orig', 'dms_dest']].drop_duplicates().itertuples(index=False, name=None))
faf_od_to_county_ratios: defaultdict[tuple[str, str], list[tuple[tuple[str, str], tuple[str, str], float]]] = defaultdict(list)
for faf_orig_zone, faf_dest_zone in faf_ods:
    total_truck_trips = 0.0
    for orig_county in faf_to_fips[faf_orig_zone]:
        for dest_county in faf_to_fips[faf_dest_zone]:
            total_truck_trips += od_fips_to_truck_trips[orig_county, dest_county]
    for orig_county in faf_to_fips[faf_orig_zone]:
        for dest_county in faf_to_fips[faf_dest_zone]:
            if total_truck_trips > 0.0:
                ratio = od_fips_to_truck_trips[orig_county, dest_county] / total_truck_trips
            else:
                ratio = 0.0
            faf_od_to_county_ratios[faf_orig_zone, faf_dest_zone].append((orig_county, dest_county, ratio))

In [43]:
print(f'Number of FAF OD pairs: {len(faf_od_to_county_ratios)}')
print(f'Number of county-to-county pairs: {sum(len(odlist) for odlist in faf_od_to_county_ratios.values())}')
print(f'Faf size: {faf.shape}')

Number of FAF OD pairs: 17412
Number of county-to-county pairs: 9934554
Faf size: (2311632, 30)


In [41]:
[ (od, len(odlist)) for od, odlist in faf_od_to_county_ratios.items()]

[(('230', '092'), 80),
 (('371', '364'), 72),
 (('339', '171'), 65),
 (('559', '131'), 2496),
 (('212', '401'), 88),
 (('201', '099'), 27),
 (('172', '329'), 135),
 (('342', '409'), 413),
 (('129', '423'), 215),
 (('401', '499'), 152),
 (('111', '292'), 8),
 (('182', '062'), 126),
 (('221', '460'), 594),
 (('221', '229'), 378),
 (('212', '423'), 55),
 (('539', '373'), 297),
 (('402', '219'), 1010),
 (('483', '179'), 480),
 (('123', '429'), 329),
 (('394', '221'), 63),
 (('092', '499'), 95),
 (('409', '172'), 531),
 (('292', '531'), 72),
 (('121', '251'), 48),
 (('423', '399'), 225),
 (('488', '280'), 664),
 (('401', '019'), 432),
 (('271', '230'), 304),
 (('122', '429'), 329),
 (('471', '451'), 9),
 (('361', '452'), 100),
 (('171', '179'), 1040),
 (('159', '379'), 280),
 (('092', '019'), 270),
 (('269', '269'), 4225),
 (('319', '472'), 1479),
 (('279', '041'), 136),
 (('419', '411'), 243),
 (('129', '123'), 301),
 (('479', '499'), 1178),
 (('452', '311'), 60),
 (('373', '471'), 33),
 (

In [ ]:
## old code that does not fit into memory

# fr_orig_list: list[str] = list()
# dms_orig_list: list[str] = list()
# dms_dest_list: list[str] = list()
# fr_dest_list: list[str] = list()
# fr_inmode_list: list[str] = list()
# dms_mode_list: list[str] = list()
# fr_outmode_list: list[str] = list()
# sctg2_list: list[str] = list()
# trade_type_list: list[str] = list()
# dist_band_list: list[str] = list()
# tons_2018_list: list[float] = list()
# tons_2019_list: list[float] = list()
# tons_2020_list: list[float] = list()
# tons_2021_list: list[float] = list()
# tons_2022_list: list[float] = list()
# value_2018_list: list[float] = list()
# value_2019_list: list[float] = list()
# value_2020_list: list[float] = list()
# value_2021_list: list[float] = list()
# value_2022_list: list[float] = list()
# current_value_2018_list: list[float] = list()
# current_value_2019_list: list[float] = list()
# current_value_2020_list: list[float] = list()
# current_value_2021_list: list[float] = list()
# current_value_2022_list: list[float] = list()
# tmiles_2018_list: list[float] = list()
# tmiles_2019_list: list[float] = list()
# tmiles_2020_list: list[float] = list()
# tmiles_2021_list: list[float] = list()
# tmiles_2022_list: list[float] = list()

# for row in faf.itertuples(index=False):
#     faf_fr_orig = row.fr_orig
#     faf_dms_orig = row.dms_orig
#     faf_dms_dest = row.dms_dest
#     faf_fr_dest = row.fr_dest
#     faf_fr_inmode = row.fr_inmode
#     faf_dms_mode = row.dms_mode
#     faf_fr_outmode = row.fr_outmode
#     faf_sctg2 = row.sctg2
#     faf_trade_type = row.trade_type
#     faf_dist_band = row.dist_band
#     for orig_county, dest_county, ratio in faf_od_to_county_ratios[faf_dms_orig, faf_dms_dest]:
#         fr_orig_list.append(faf_fr_orig)
#         dms_orig_list.append(faf_dms_orig)
#         dms_dest_list.append(faf_dms_dest)
#         fr_dest_list.append(faf_fr_dest)
#         fr_inmode_list.append(faf_fr_inmode)
#         dms_mode_list.append(faf_dms_mode)
#         fr_outmode_list.append(faf_fr_outmode)
#         sctg2_list.append(faf_sctg2)
#         trade_type_list.append(faf_trade_type)
#         dist_band_list.append(faf_dist_band)
#         tons_2018_list.append(ratio * row.tons_2018)
#         tons_2019_list.append(ratio * row.tons_2019)
#         tons_2020_list.append(ratio * row.tons_2020)
#         tons_2021_list.append(ratio * row.tons_2021)
#         tons_2022_list.append(ratio * row.tons_2022)
#         value_2018_list.append(ratio * row.value_2018)
#         value_2019_list.append(ratio * row.value_2019)
#         value_2020_list.append(ratio * row.value_2020)
#         value_2021_list.append(ratio * row.value_2021)
#         value_2022_list.append(ratio * row.value_2022)
#         current_value_2018_list.append(ratio * row.current_value_2018)
#         current_value_2019_list.append(ratio * row.current_value_2019)
#         current_value_2020_list.append(ratio * row.current_value_2020)
#         current_value_2021_list.append(ratio * row.current_value_2021)
#         current_value_2022_list.append(ratio * row.current_value_2022)
#         tmiles_2018_list.append(ratio * row.tmiles_2018)
#         tmiles_2019_list.append(ratio * row.tmiles_2019)
#         tmiles_2020_list.append(ratio * row.tmiles_2020)
#         tmiles_2021_list.append(ratio * row.tmiles_2021)
#         tmiles_2022_list.append(ratio * row.tmiles_2022)

# faf_with_counties_df = pd.DataFrame({
#     "fr_orig": fr_orig_list,
#     "dms_orig": dms_orig_list,
#     "dms_dest": dms_dest_list,
#     "fr_dest": fr_dest_list,
#     "fr_inmode": fr_inmode_list,
#     "dms_mode": dms_mode_list,
#     "fr_outmode": fr_outmode_list,
#     "sctg2": sctg2_list,
#     "trade_type": trade_type_list,
#     "dist_band": dist_band_list,
#     "tons_2018": tons_2018_list,
#     "tons_2019": tons_2019_list,
#     "tons_2020": tons_2020_list,
#     "tons_2021": tons_2021_list,
#     "tons_2022": tons_2022_list,
#     "value_2018": value_2018_list,
#     "value_2019": value_2019_list,
#     "value_2020": value_2020_list,
#     "value_2021": value_2021_list,
#     "value_2022": value_2022_list,
#     "current_value_2018": current_value_2018_list,
#     "current_value_2019": current_value_2019_list,
#     "current_value_2020": current_value_2020_list,
#     "current_value_2021": current_value_2021_list,
#     "current_value_2022": current_value_2022_list,
#     "tmiles_2018": tmiles_2018_list,
#     "tmiles_2019": tmiles_2019_list,
#     "tmiles_2020": tmiles_2020_list,
#     "tmiles_2021": tmiles_2021_list,
#     "tmiles_2022": tmiles_2022_list
# })

In [35]:
# Too big to fit in memory, so output one line at a time in a CSV file.  No need to use the csv writer for this

record_counter = 0
with open('faf5_by_county.csv', "w") as csv_file:
    csv_file.write("fr_orig,dms_orig,st_fips_orig,st_county_orig,dms_dest,st_fips_dest,county_fips_dest,fr_dest,"
                   "fr_inmode,dms_mode,fr_outmode,sctg2,trade_type,dist_band,"
                   "tons_2018,tons_2019,tons_2020,tons_2021,tons_2022,"
                   "value_2018,value_2019,value_2020,value_2021,value_2022,"
                   "current_value_2018,current_value_2019,current_value_2020,current_value_2021,current_value_2022,"
                   "tmiles_2018,tmiles_2019,tmiles_2020,tmiles_2021,tmiles_2022"
                   "\n")

    for row in faf.itertuples(index=False):
        faf_fr_orig = row.fr_orig
        faf_dms_orig = row.dms_orig
        faf_dms_dest = row.dms_dest
        faf_fr_dest = row.fr_dest
        faf_fr_inmode = row.fr_inmode
        faf_dms_mode = row.dms_mode
        faf_fr_outmode = row.fr_outmode
        faf_sctg2 = row.sctg2
        faf_trade_type = row.trade_type
        faf_dist_band = row.dist_band
        for orig_county, dest_county, ratio in faf_od_to_county_ratios[faf_dms_orig, faf_dms_dest]:
            record_counter += 1
            if 0 == record_counter % 50000:
                pass
            s = (f'"{faf_fr_orig}","{faf_dms_orig}","{orig_county[0]}","{orig_county[1]}",'
                f'"{faf_dms_dest}","{dest_county[0]}","{dest_county[1]}","{faf_fr_dest}",'
                f'"{faf_fr_inmode}","{faf_dms_mode}","{faf_fr_outmode}","{faf_sctg2}","{faf_trade_type}","{faf_dist_band}",'
                f'{ratio * row.tons_2018},{ratio * row.tons_2019},{ratio * row.tons_2020},{ratio * row.tons_2021},{ratio * row.tons_2022},'
                f'{ratio * row.value_2018},{ratio * row.value_2019},{ratio * row.value_2020},{ratio * row.value_2021},{ratio * row.value_2022},'
                f'{ratio * row.current_value_2018},{ratio * row.current_value_2019},{ratio * row.current_value_2020},{ratio * row.current_value_2021},{ratio * row.current_value_2022},'
                f'{ratio * row.tmiles_2018},{ratio * row.tmiles_2019},{ratio * row.tmiles_2020},{ratio * row.tmiles_2021},{ratio * row.tmiles_2022}'
                f'\n'
            )
            csv_file.write(s)


KeyboardInterrupt: 